# Spark Play 01

## Install and setup

Setup instructions for running pyspark in jupyter notebooks (using conda environment, such as "my-env"):
```
java -version # Should be 1.8.0_241 ... i.e. Final Java 8 release 
conda create -n my-env
conda activate -n my-env python=3.7 jupyter
conda install -c conda-forge pyspark
conda install -c anaconda ipykernel
python -m ipykernel install --user --name=my-env
jupyter notebook
```
Open new notebook file by selecting New > my-env

In [1]:
import numpy as np
import pandas as pd


from pyspark import SparkContext
from pyspark.sql import SparkSession

import pyspark.sql.functions as psf

First create a `SparkContext` which connects the Spark application to the cluster (in this case local).

In [2]:
sc = SparkContext('local[*]')
print(sc)
print(sc.version)

<SparkContext master=local[*] appName=pyspark-shell>
2.4.5


Then get or create a new `SparkSession` which we'll use to control the Spark driver.

In [3]:
spark = SparkSession.builder.getOrCreate()
print(spark)

List the contents of the Spark session using its `catalog` attribute, which has methods such as `listTables()`. (Currently nothing in the session.)

In [4]:
spark.catalog.listTables()

[]

Other useful `catalog` methods might include:
- `cacheTable(tableName)`/`uncacheTable(tableName)`/`isCached(tableName)`
- `createTable()`
- `listColumns(tableName)`
- `listFunctions(dbName=None)` -- functions registered in a specified database.

-------------------------------------

## Using spark driver for first time

We can start off generating sequences to play with using the session:

In [5]:
spark.range(1000) # Returns dataframe

DataFrame[id: bigint]

In [6]:
spark.range(1000).toDF("number") # Give it column header 'number'

DataFrame[number: bigint]

We could assign that dataframe to a variable and work on that. Or we can keep chaining functions, using functional programming style, to minimise use of intermediate global objects and variables.

In [7]:
spark.range(1000)\
    .toDF("number")\
    .where("number % 2 = 0")\
    .count()

500

The executions we've run above are saved in the Spark application as previous "jobs". You can view the job history, among other things, by looking at the Spark UI, on port 4040 of the driver node, which in our local case is http://localhost:4040.

-----------------------

## Running first queries on a dataset 

We can run SQL queries or dataframe manipulations through the `SparkSession`. But first we need to read some data into the session, using its `read` methods.

In [8]:
flights = spark.read.csv("./data/flights.csv", inferSchema=True, header=True)

In [9]:
flights.columns

['YEAR',
 'MONTH',
 'DAY',
 'DAY_OF_WEEK',
 'AIRLINE',
 'FLIGHT_NUMBER',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'SCHEDULED_DEPARTURE',
 'DEPARTURE_TIME',
 'DEPARTURE_DELAY',
 'TAXI_OUT',
 'WHEELS_OFF',
 'SCHEDULED_TIME',
 'ELAPSED_TIME',
 'AIR_TIME',
 'DISTANCE',
 'WHEELS_ON',
 'TAXI_IN',
 'SCHEDULED_ARRIVAL',
 'ARRIVAL_TIME',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'CANCELLATION_REASON',
 'AIR_SYSTEM_DELAY',
 'SECURITY_DELAY',
 'AIRLINE_DELAY',
 'LATE_AIRCRAFT_DELAY',
 'WEATHER_DELAY']

Here's a dataframe manipulation chain, finding the number of flights within each origin-destination group.

In [10]:
flights\
    .select('FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 
        'SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL',
        'DEPARTURE_TIME', 'ARRIVAL_TIME', 'ARRIVAL_DELAY')\
    .groupBy('ORIGIN_AIRPORT', 'DESTINATION_AIRPORT')\
    .count()\
    .withColumnRenamed('count', 'total_flights')\
    .sort(psf.desc('total_flights'))\
    .take(10) # For comparison with sql code, could be written as .limit(10)/.collect()

[Row(ORIGIN_AIRPORT='SFO', DESTINATION_AIRPORT='LAX', total_flights=13744),
 Row(ORIGIN_AIRPORT='LAX', DESTINATION_AIRPORT='SFO', total_flights=13457),
 Row(ORIGIN_AIRPORT='JFK', DESTINATION_AIRPORT='LAX', total_flights=12016),
 Row(ORIGIN_AIRPORT='LAX', DESTINATION_AIRPORT='JFK', total_flights=12015),
 Row(ORIGIN_AIRPORT='LAS', DESTINATION_AIRPORT='LAX', total_flights=9715),
 Row(ORIGIN_AIRPORT='LGA', DESTINATION_AIRPORT='ORD', total_flights=9639),
 Row(ORIGIN_AIRPORT='LAX', DESTINATION_AIRPORT='LAS', total_flights=9594),
 Row(ORIGIN_AIRPORT='ORD', DESTINATION_AIRPORT='LGA', total_flights=9575),
 Row(ORIGIN_AIRPORT='SFO', DESTINATION_AIRPORT='JFK', total_flights=8440),
 Row(ORIGIN_AIRPORT='JFK', DESTINATION_AIRPORT='SFO', total_flights=8437)]

The equivalent can be written as a SQL query, and from Spark's point of view they are an identical implementation. To run SQL queries we must first convert the dataframe into a database table. We then query the table by passing SQL code to the session driver.

In [11]:
flights.createOrReplaceTempView("flights")

In [12]:
spark.sql("""
    FROM flights
        SELECT origin_airport, destination_airport, count(*) AS total_flights
        GROUP BY origin_airport, destination_airport
        ORDER BY total_flights DESC
        LIMIT 10
    """).collect()

[Row(origin_airport='SFO', destination_airport='LAX', total_flights=13744),
 Row(origin_airport='LAX', destination_airport='SFO', total_flights=13457),
 Row(origin_airport='JFK', destination_airport='LAX', total_flights=12016),
 Row(origin_airport='LAX', destination_airport='JFK', total_flights=12015),
 Row(origin_airport='LAS', destination_airport='LAX', total_flights=9715),
 Row(origin_airport='LGA', destination_airport='ORD', total_flights=9639),
 Row(origin_airport='LAX', destination_airport='LAS', total_flights=9594),
 Row(origin_airport='ORD', destination_airport='LGA', total_flights=9575),
 Row(origin_airport='SFO', destination_airport='JFK', total_flights=8440),
 Row(origin_airport='JFK', destination_airport='SFO', total_flights=8437)]

Note: You don't have to cap off SQL queries with `;`.

**Note also:** The `take()` and `collect()` methods are actions at the end of spark queries which will **load the output data into the driver's memory**.

We can take the queried data using either SQL or DataFrame styles, and if we create a local `pandas` copy using `.toPandas()`

In [13]:
flights\
    .select('FLIGHT_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 
        'SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL',
        'DEPARTURE_TIME', 'ARRIVAL_TIME', 'ARRIVAL_DELAY')\
    .groupBy('ORIGIN_AIRPORT', 'DESTINATION_AIRPORT')\
    .avg('ARRIVAL_DELAY')\
    .withColumnRenamed('avg(ARRIVAL_DELAY)', 'avg_delay')\
    .sort(psf.desc('avg_delay'))\
    .limit(20)\
    .toPandas()

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,avg_delay
0,IAD,TTN,381.000000
1,SWF,PBI,260.500000
2,RIC,CAE,228.000000
3,RDU,IND,208.000000
4,10581,11618,163.000000
5,FCA,MSO,148.000000
6,SWF,RSW,140.000000
7,10581,12953,138.333333
8,14843,12264,122.000000
9,OAK,FLL,106.000000


Let's do one more for luck...

In [14]:
spark.sql("""
SELECT origin_airport, destination_airport, airline, ROUND(AVG(air_time) / 60, 2) AS duration_hrs
FROM flights
GROUP BY origin_airport, destination_airport, airline
ORDER BY duration_hrs DESC
""").toPandas()

,origin_airport,destination_airport,airline,duration_hrs
0,JFK,HNL,DL,10.74
1,JFK,HNL,HA,10.58
2,12478,12173,HA,10.46
3,EWR,HNL,UA,10.21
4,11618,12173,UA,10.02
...,...,...,...,...
14735,CWA,ATW,EV,NaN
14736,MCO,HPN,EV,NaN
14737,GCC,RAP,OO,NaN
14738,FSD,LNK,EV,NaN


-------------------------------------

## Manipulating Spark DataFrames

### Note on immutability in Spark

Spark DataFrames are immutable, because you're supposed to leave the data as-is and do all the transformations in your code. If you wanted to update your DataFrame, you'd have to re-assign it.

For example, let's say we needed to add an extra column. We could use the Spark DataFrame's `withColumn()` method like so:

In [15]:
flights = flights.withColumn('duration_hrs', flights.AIR_TIME / 60)
'duration_hrs' in flights.columns

True

### Filter

The `filter()` method takes on of two options:
- A string, which represents what would follow a WHERE clause in SQL, or
- A Spark Column of boolean (True/False) values

In other words, the following two lines are equivalent.

In [16]:
print(flights.filter("air_time > 120"), "\n")
print(flights.filter(flights.AIR_TIME > 120))

DataFrame[YEAR: int, MONTH: int, DAY: int, DAY_OF_WEEK: int, AIRLINE: string, FLIGHT_NUMBER: int, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: int, DEPARTURE_TIME: int, DEPARTURE_DELAY: int, TAXI_OUT: int, WHEELS_OFF: int, SCHEDULED_TIME: int, ELAPSED_TIME: int, AIR_TIME: int, DISTANCE: int, WHEELS_ON: int, TAXI_IN: int, SCHEDULED_ARRIVAL: int, ARRIVAL_TIME: int, ARRIVAL_DELAY: int, DIVERTED: int, CANCELLED: int, CANCELLATION_REASON: string, AIR_SYSTEM_DELAY: int, SECURITY_DELAY: int, AIRLINE_DELAY: int, LATE_AIRCRAFT_DELAY: int, WEATHER_DELAY: int, duration_hrs: double] 

DataFrame[YEAR: int, MONTH: int, DAY: int, DAY_OF_WEEK: int, AIRLINE: string, FLIGHT_NUMBER: int, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: int, DEPARTURE_TIME: int, DEPARTURE_DELAY: int, TAXI_OUT: int, WHEELS_OFF: int, SCHEDULED_TIME: int, ELAPSED_TIME: int, AIR_TIME: int, DISTANCE: int, WHEELS_ON: int, TAXI_IN: in

**Note:** That when using string approach, the variable names are **not case sensitive**, just like with SQL queries. But when writing in Python code, they are.

### Select

The `select()` method takes multiple arguments that define which columns to select, either as strings of column names or the column objects themselves.

In [17]:
flights.select("arrival_time", flights.DEPARTURE_TIME)

DataFrame[arrival_time: int, DEPARTURE_TIME: int]

**Note again:** That when selecting columns using strings, they are not case sensitive, just like in SQL. But passing a column object is case sensitive.

As we saw just a moment above, you can perform operations on your columns to transform them using the `withColumn()` method, which returns all columns in the outputted DataFrame.

But you can also do this inside `select()`, one of two ways:
- Pass it a column object which we transform, with the option of renaming using the `alias()` method.
- Pass it strings, which represent SQL expressions we'd pass to `SELECT`, with the option of renaming using `AS`. **But to do it this way, you have to use `selectExpr()` instead of `select()`.**

In [18]:
flights.select("arrival_time", flights.DEPARTURE_TIME, (flights.AIR_TIME / 60).alias('duration_hrs'))

DataFrame[arrival_time: int, DEPARTURE_TIME: int, duration_hrs: double]

In [19]:
flights.selectExpr("arrival_time", "departure_time", "air_time / 60 AS duration_hrs")

DataFrame[arrival_time: int, departure_time: int, duration_hrs: double]

Another example:

In [20]:
flights.select("origin_airport", "destination_airport", "tail_number",
               (flights.DISTANCE/(flights.AIR_TIME/60)).alias("avg_speed"))

DataFrame[origin_airport: string, destination_airport: string, tail_number: string, avg_speed: double]

In [21]:
flights.selectExpr("origin_airport", "destination_airport", "tail_number",
                   "distance/(air_time/60) as avg_speed")

DataFrame[origin_airport: string, destination_airport: string, tail_number: string, avg_speed: double]

### GroupBy Aggregate

We can group our DataFrame using `groupBy()`. Once we've created a grouped DataFrame (GroupedData), we can use the GroupedDat methods for aggregation, like `min()`, `max()`, `count()`, `avg()`, and so on. We can aggregate the whole data by leaving `groupBy()` empty.

Average flight duration:

In [22]:
flights\
    .select((flights.AIR_TIME / 60).alias('duration_hrs'))\
    .groupBy()\
    .avg('duration_hrs')\
    .show()

+------------------+
| avg(duration_hrs)|
+------------------+
|1.8918604681687405|
+------------------+



Average flight duration by airline carrier, sorted in descending order:

In [23]:
flights\
    .selectExpr("airline", "air_time / 60 AS duration_hrs")\
    .groupBy('airline')\
    .avg('duration_hrs')\
    .sort(psf.desc('avg(duration_hrs)'))\
    .show(5)

# ^^ Note: See agg() below for renaming var at same time

+-------+------------------+
|airline| avg(duration_hrs)|
+-------+------------------+
|     VX| 3.043846601793797|
|     UA|2.7472647618372408|
|     AS|2.6307464073713316|
|     B6| 2.397641535835225|
|     AA|2.3301034689931734|
+-------+------------------+
only showing top 5 rows



Total number of flights taken, by airline and origin airport, sorted in descending order:

In [24]:
flights\
    .groupBy('origin_airport', 'airline')\
    .count()\
    .sort(psf.desc('count'))\
    .show(5)

+--------------+-------+------+
|origin_airport|airline| count|
+--------------+-------+------+
|           ATL|     DL|221705|
|           DFW|     AA|134270|
|           MDW|     WN| 76350|
|           LAS|     WN| 68520|
|           BWI|     WN| 64063|
+--------------+-------+------+
only showing top 5 rows



#### `agg()` method

As well as the basic aggregation functions, there is also the generic `agg()` method, which lets us pass any of the aggregation functions from the `pyspark.sql.functions` submodule. (I loaded this submodule earlier as `psf`.)

Every function in the submodule takes the name of a GroupedData column as its argument.

Here are some examples:

In [25]:
flights\
    .groupBy('month', 'destination_airport')\
    .agg((psf.stddev('departure_delay')).alias('dep_delay_stdev'))\
    .show(5)

+-----+-------------------+------------------+
|month|destination_airport|   dep_delay_stdev|
+-----+-------------------+------------------+
|    1|                ACY| 32.81265545745788|
|    1|                EYW| 63.65052137951282|
|    1|                OME|19.546923118225806|
|    1|                RDM|34.260918828983726|
|    1|                TWF| 7.985009333135624|
+-----+-------------------+------------------+
only showing top 5 rows



Note: We can use the `alias()` method with aggregation as well as when selecting columns as we saw earlier. But I don't think aliasing works when using the basic aggregators; you have to wrap it inside `agg()` instead.

### Joins

In [26]:
airlines = spark.read.csv("./data/airlines.csv", inferSchema=True, header=True)
airlines.show(5)

+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
+---------+--------------------+
only showing top 5 rows



In [27]:
airports = spark.read.csv("./data/airports.csv", inferSchema=True, header=True)
airports.show(5)

+---------+--------------------+-----------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|       CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-----------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|  Allentown|   PA|    USA|40.65236|  -75.4404|
|      ABI|Abilene Regional ...|    Abilene|   TX|    USA|32.41132|  -99.6819|
|      ABQ|Albuquerque Inter...|Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|   Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|     Albany|   GA|    USA|31.53552| -84.19447|
+---------+--------------------+-----------+-----+-------+--------+----------+
only showing top 5 rows



The `join()` method lets us do all kinds of joins. First arg is the dataset we want to use in our join, second arg `on` specifies the keys to join on, and third arg `how` specifies which kind of join to perform.

In [28]:
flights\
    .withColumnRenamed("destination_airport", "iata_code")\
    .join(airports, on="iata_code", how="leftouter")\
    .show(5)

+---------+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+-----------------+--------------------+---------------+-----+-------+--------+----------+
|iata_code|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|     duration_hrs|             AIRPORT|           CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+----+-----+---+-----------+-------+--